In [16]:
!pip install -qU langgraph langchain_google_genai langchain_core

In [17]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_core.tools import tool
from google.colab import userdata

In [19]:
os.environ["GEMINI_API_KEY"] = userdata.get('GEMINI_API_KEY')

In [20]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.3)

In [21]:
@tool
def search_internet(query: str) -> str:
    """
    Use this tool to search the internet for up-to-date information.
    The input is the search query itself.
    """

    print(f"\nTOOL CALLED: Searching for '{query}' \n")
    if "latest langgraph" in query.lower():
        return "The latest official documentation on LangGraph confirms the use of a `messages` list in the State, and recommends using `StateGraph` for custom agent flows."
    return f"Information about '{query}' is available."


llm_with_tools = llm.bind_tools([search_internet])

print("Setup complete. LLM and Tool initialized.")

Setup complete. LLM and Tool initialized.


In [22]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
import operator


class AgentState(TypedDict):
    # 'messages' is the key that holds the conversation history.
    # Annotated[..., operator.add] means new lists of messages are appended to the existing list.
    messages: Annotated[List[BaseMessage], operator.add]

print("AgentState defined: It stores all 'messages' (conversation history).")

AgentState defined: It stores all 'messages' (conversation history).


In [23]:
# Node 1: The Decision Maker (LLM)
def run_agent(state: AgentState) -> dict:
    """Invokes the LLM to get a response or a tool call."""
    print("NODE: Running Agent (Decision Making)")
    messages = state["messages"]


    result = llm_with_tools.invoke(messages)


    return {"messages": [result]}

# Node 2: The Action Taker (Tool Executor)
def execute_tool(state: AgentState) -> dict:
    """Executes the tool call requested by the LLM."""
    print("NODE: Executing Tool")


    tool_calls = state["messages"][-1].tool_calls


    first_tool_call = tool_calls[0]
    tool_name = first_tool_call["name"]
    tool_args = first_tool_call["args"]


    if tool_name == "search_internet":

        tool_output = search_internet.func(**tool_args)
    else:
        tool_output = f"Unknown tool: {tool_name}"


    tool_message = ToolMessage(
        content=tool_output,
        name=tool_name,

        tool_call_id=first_tool_call["id"],
    )


    return {"messages": [tool_message]}

In [24]:
from langgraph.graph import StateGraph, END


def decide_next_step(state: AgentState) -> str:
    """
    Decides whether to execute a tool or end the conversation based on the LLM's response.
    Returns: "continue" (to tool node) or "end" (to final answer).
    """
    last_message = state["messages"][-1]


    if last_message.tool_calls:

        print("EDGE: Tool call detected. Moving to 'tools' node.")
        return "tools"
    else:
        print("EDGE: Final answer detected. Ending graph execution.")
        return "end"



In [26]:
from langgraph.graph import StateGraph, END

workflow = StateGraph(AgentState)

workflow.add_node("agent", run_agent)
workflow.add_node("tools", execute_tool)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    decide_next_step,
    {
        "tools": "tools",
        "end": END
    }
)

workflow.add_edge('tools', 'agent')

app = workflow.compile()

In [28]:
from langchain_core.messages import HumanMessage

initial_state = {"messages": [HumanMessage(content="What is the latest guidance for building agents with LangGraph?")]}

print("Agent Execution Stream Started")

print("-" * 30)
for step in app.stream(initial_state):
    node_name = list(step.keys())[0]
    print(f"-> Node executed: **{node_name}**")

final_state = app.invoke(initial_state)

final_messages = final_state["messages"]

print("-" * 30)
print("\n FINAL ANSWER from Agent")
print(final_messages[-1].content)
print("-" * 30)

Agent Execution Stream Started
------------------------------
--- NODE: Running Agent (Decision Making) ---
EDGE: Tool call detected. Moving to 'tools' node.
-> Node executed: **agent**
--- NODE: Executing Tool ---

--- TOOL CALLED: Searching for 'latest guidance for building agents with LangGraph' ---

-> Node executed: **tools**
--- NODE: Running Agent (Decision Making) ---
EDGE: Final answer detected. Ending graph execution.
-> Node executed: **agent**
--- NODE: Running Agent (Decision Making) ---
EDGE: Tool call detected. Moving to 'tools' node.
--- NODE: Executing Tool ---

--- TOOL CALLED: Searching for 'LangGraph agent building latest guidance' ---

--- NODE: Running Agent (Decision Making) ---
EDGE: Final answer detected. Ending graph execution.
------------------------------

 FINAL ANSWER from Agent
[{'type': 'text', 'text': "I found that information about LangGraph agent building guidance is available. LangGraph is a library that helps build robust and stateful multi-actor a